In [1]:
import pandas as pd

import nltk
from nltk.metrics.agreement import AnnotationTask

In [2]:
annotations = pd.read_csv('Annotation_Mastersheet.csv')  

annotations.head()

,question,answer,Ahsan,Unnamed: 3,Unnamed: 4,Oksana,Unnamed: 6,Unnamed: 7,Ruoyao,Unnamed: 9,Unnamed: 10,Jeremy,Unnamed: 12,Unnamed: 13
0,What is the use of the yield keyword in Python...,"To understand what yield does, you must unders...",iterator,beginner,direct,iterator,beginner,direct,objects,NaN,NaN,NaN,NaN,NaN
1,"Given the following code, what does the if __n...",It's boilerplate code that protects users from...,classes,beginner,direct,built-in,beginner,direct,variables,NaN,NaN,NaN,NaN,NaN
2,"Given the following code, what does the if __n...","if __name__ == ""__main__"" is the part that run...",classes,beginner,direct,built-in,beginner,direct,variables,NaN,NaN,NaN,NaN,NaN
3,"In Python, what are metaclasses and what do we...","Before understanding metaclasses, you need to ...",classes,beginner,direct,classes,beginner,direct,classes,NaN,NaN,NaN,NaN,NaN
4,"In Python, what are metaclasses and what do we...",A metaclass is the class of a class. A class d...,classes,beginner,direct,classes,beginner,direct,classes,NaN,NaN,NaN,NaN,NaN


In [3]:
#[No. 0-250] Ahsan & Oksana
#[No. 250-500] Ahsan & Ruoyao
#[No. 500-750] Oksana & Jeremy
#[No. 750-1000] Ruoyao & Jeremy
#Since two people annotated each question-answer pair, I am making two columns called ' Annotator 1' and 'Annotator 2' instead of having several with many NaN values


ahsan = annotations.iloc[0:500, 2:5].rename(columns={'Ahsan':'Classification_1', 'Unnamed: 3': 'Difficulty_1', 'Unnamed: 4': 'Directness_1'})
oksana1 = annotations.iloc[500:750, 5:8].rename(columns={'Oksana':'Classification_1', 'Unnamed: 6': 'Difficulty_1', 'Unnamed: 7': 'Directness_1'})
ruoyao1 = annotations.iloc[750:1000, 8:11].rename(columns={'Ruoyao':'Classification_1', 'Unnamed: 9': 'Difficulty_1', 'Unnamed: 10': 'Directness_1'})

jeremy = annotations.iloc[500:1000, 11:14].rename(columns={'Jeremy':'Classification_2', 'Unnamed: 12': 'Difficulty_2', 'Unnamed: 13': 'Directness_2'})
oksana2 = annotations.iloc[0:250, 5:8].rename(columns={'Oksana':'Classification_2', 'Unnamed: 6': 'Difficulty_2', 'Unnamed: 7': 'Directness_2'})
ruoyao2 = annotations.iloc[250:500, 8:11].rename(columns={'Ruoyao':'Classification_2', 'Unnamed: 9': 'Difficulty_2', 'Unnamed: 10': 'Directness_2'})


anno1 = pd.concat([ahsan, oksana1, ruoyao1])
anno2 = pd.concat([oksana2, ruoyao2, jeremy])

annos = pd.concat([anno1, anno2], axis = 1)

annos["QA"] = 'Q: ' + annotations["question"] + ' A: ' + annotations["answer"]

annos = annos.dropna()
annos['Directness_1'] = annos['Directness_1'].str.strip()
annos['Classification_1'] = annos['Classification_1'].str.strip()
annos['Classification_2'] = annos['Classification_2'].str.strip()

In [4]:
annos

,Classification_1,Difficulty_1,Directness_1,Classification_2,Difficulty_2,Directness_2,QA
0,iterator,beginner,direct,iterator,beginner,direct,Q: What is the use of the yield keyword in Pyt...
1,classes,beginner,direct,built-in,beginner,direct,"Q: Given the following code, what does the if ..."
2,classes,beginner,direct,built-in,beginner,direct,"Q: Given the following code, what does the if ..."
3,classes,beginner,direct,classes,beginner,direct,"Q: In Python, what are metaclasses and what do..."
4,classes,beginner,direct,classes,beginner,direct,"Q: In Python, what are metaclasses and what do..."
...,...,...,...,...,...,...,...
994,function,advanced,direct,function,advanced,direct,Q: I've recently become interested in algorith...
995,error handling,intermediate,direct,file I/O,intermediate,direct,Q: Say I have a function: A: I'd recommend rea...
996,error handling,intermediate,direct,file I/O,intermediate,direct,Q: Say I have a function: A: First add the # -...
997,data type,beginner,direct,libraries,beginner,direct,Q: I have a DataFrame with four columns. I wan...


In [5]:
def convert_to_triples(df, anno1, anno2):
    triple_list = []
    for index, row in df.iterrows():
        triple_list.append((anno1, row['QA'], row[anno1]))
        triple_list.append((anno2, row['QA'], row[anno2]))
    return triple_list

classification_triples = convert_to_triples(annos, 'Classification_1', 'Classification_2')
difficulty_triples = convert_to_triples(annos, 'Difficulty_1', 'Difficulty_2')
directness_triples = convert_to_triples(annos, 'Directness_1', 'Directness_2')

In [6]:
annotation_task_class = AnnotationTask(classification_triples)
annotation_task_diff = AnnotationTask(difficulty_triples)
annotation_task_dir = AnnotationTask(directness_triples)

In [7]:
classification = round(annotation_task_class.avg_Ao(), 2)
difficulty = round(annotation_task_diff.avg_Ao(), 2)
directness = round(annotation_task_dir.avg_Ao(), 2)

print('Average Annotator Agreement')
print('Classification: ', classification)
print('Difficulty: ', difficulty)
print('Directness: ', directness)
print('Average: ', round((classification + difficulty + directness) / 3, 2))

Average Annotator Agreement
Classification:  0.58
Difficulty:  0.73
Directness:  0.92
Average:  0.74


In [8]:
classification = round(annotation_task_class.S(), 2)
difficulty = round(annotation_task_diff.S(), 2)
directness = round(annotation_task_dir.S(), 2)

print('The Simplest Assumption')
print('Classification: ', classification)
print('Difficulty: ', difficulty)
print('Directness: ', directness)
print('Average: ', round((classification + difficulty + directness) / 3, 2))

The Simplest Assumption
Classification:  0.56
Difficulty:  0.66
Directness:  0.87
Average:  0.7


In [9]:
classification = round(annotation_task_class.pi(), 2)
difficulty = round(annotation_task_diff.pi(), 2)
directness = round(annotation_task_dir.pi(), 2)

print("Scott's Pi")
print('Classification: ', classification)
print('Difficulty: ', difficulty)
print('Directness: ', directness)
print('Average: ', round((classification + difficulty + directness) / 3, 2))

Scott's Pi
Classification:  0.54
Difficulty:  0.45
Directness:  0.02
Average:  0.34


In [10]:
classification = round(annotation_task_class.kappa(), 2)
difficulty = round(annotation_task_diff.kappa(), 2)
directness = round(annotation_task_dir.kappa(), 2)

print("Cohen's Kappa")
print('Classification: ', classification)
print('Difficulty: ', difficulty)
print('Directness: ', directness)
print('Average: ', round((classification + difficulty + directness) / 3, 2))

Cohen's Kappa
Classification:  0.54
Difficulty:  0.45
Directness:  0.02
Average:  0.34
